In [ ]:
import pandas as pd
import numpy as np
import random
from smart import Smart

real_energy_usages = pd.read_csv("EUR/Integrated/energy_usages.csv")

smart = Smart()
print(real_energy_usages["building_name"].value_counts())
# map IDs to building names, to do:use real mappings
id_to_name = {
    "0599100000309497" : "Polak",
    "0599100000319364" : "Sanders en PG",
    "0599100000332174" : "Bayle",
    "0599100000332175" : "Mandeville",
    "0599100000332176" : "Van der Groot"
}

In [ ]:
def simulateEnergyUsage():
    building_energy_mu = random.randint(5, 30)
    building_energy_sigma = 2
    building_energy_increase_constant = 1 / 100 * 30

    years = 100
    start_year = 1922

    usage = list(map(lambda x: {
        "year": x,
        "energy": np.random.normal(building_energy_mu + (x - start_year) * building_energy_increase_constant, building_energy_sigma)
        }, range(start_year, start_year + years)))
    return usage


In [ ]:
import json


# Opening JSON file
f = open('BAG_WFS_build_4326.geojson',)
data = json.load(f)

# Closing file
f.close()

first = data['features'][0]

fakes = []
for building in data["features"]:
	fake = {}
	fake["id"] = building["properties"]["identificatie"]
	fake["year"] = building["properties"]["bouwjaar"]
	fake["geometry"] = building["geometry"]

	usage = simulateEnergyUsage()


	fake["usage"] = usage
	fakes.append(fake)

# Normalization of data
energies = list(map(lambda x: list(map(lambda usage: usage["energy"], x["usage"])), fakes))
energies = [e for es in energies for e in es]
max_energy = max(energies)
min_energy = min(energies)


for fake in fakes:
	fake["year"] = (fake["year"] - 1873) / 148 
	for x in fake["usage"]:
		old_energy = x["energy"]
		x["energy"] = (x["energy"] - min_energy) / max_energy
	
print(min_energy, max_energy)
# print(fakes[4])

# Consider changing format two one longer array instead of different dictionaries

In [ ]:
with open("fakeBuildingData.json", "w") as f:
    json.dump(fakes, f)